In [1]:
from lenskit.algorithms import als
from lenskit import batch, crossfold as xf
from lenskit.metrics.predict import rmse
import pandas as pd
import joblib

from mlcomp.data.load import load_regression_train

In [2]:
df = load_regression_train()
# remove everything before 01-01-2016
#df = df.loc[df['timestamp'] > 1451602800]
#df = df[:10000]

windows = pd.DataFrame(columns=['start', 'stop'])

current_start = df['timestamp'].min()
window_size = 63072000 * 1000 # 2 years
window_offset = 47304000 * 1000 # 1 and 1/2 years
counter = 0
while current_start < df['timestamp'].max():
    windows.loc[counter] = [current_start, current_start + window_size]
    current_start += window_offset
    counter += 1


In [3]:

results = []

for index, win in windows.iterrows():

    start_t = win['start']
    stop_t = win['stop']

    data_in_window = df.loc[(df['timestamp'] > start_t) & (df['timestamp'] < stop_t)]

    ttpair_gen = xf.partition_users(data_in_window, 2, xf.SampleFrac(0.15))
    ttpair = ttpair_gen.__next__()

    train = ttpair.train
    test = ttpair.test

    # the partition method sometimes produces an empty test set if we are unlucky,
    # so we just skip if thats the case:
    if len(test) == 0:
        print(f'Skipped window {index}')
        continue

    bmf = als.BiasedMF(50)

    bmf.fit(test)

    X_val = test.drop(columns=['rating'])
    y_val = test['rating']

    pred = batch.predict(bmf, X_val)

    # lenskit messed up the index but keeps the order, so we can fix it like this:
    pred.index = X_val.index
    error = rmse(pred['prediction'], y_val)

    results.append((start_t, stop_t, bmf, error))

    print(f'Done with window {index}')

joblib.dump(results, 'time_window_result.joblib')

could not load LIBBLAS: Could not find module 'libblas' (or one of its dependencies). Try using the full path with constructor syntax.


Skipped window 0
Done with window 1
Done with window 2
Done with window 3
Done with window 4
Done with window 5
Done with window 6
Done with window 7
Done with window 8
Done with window 9


['time_window_result.joblib']

## Results:

